In [19]:
#  Подключаю необходимые библиотеки
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

In [111]:
import os
os.chdir("/home/vladimir/jupyter/pyDA/Lab_01")


In [112]:
df = pd.read_csv('winequality-red.csv', sep=';')

In [113]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [114]:
df.describe(include='all')

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


In [115]:
df['quality'].value_counts()
#несбалансированная выборка

5    681
6    638
7    199
4     53
8     18
3     10
Name: quality, dtype: int64

In [131]:
#объединим
df['quality'] = df['quality'].replace(to_replace=3, value=4).replace(8, 7).replace(4, 5)


In [132]:
df['quality'].value_counts()


5    744
6    638
7    217
Name: quality, dtype: int64

In [133]:
X = df.iloc[:, :-1].values  
y = df.iloc[:, 11].values  

In [134]:
#  разделяю на обучающую и тестовую выборку

from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)  


In [135]:
# Инициализируем модель

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42,
                               # число деревьев в лесу
                               n_estimators=80,
                               # функция для impurity ('gini' или 'entropy')
                               criterion='gini',
                               # то ли число слоев, то ли число расщеплений  
                               #  ?????????????????  но не число узлов
                               max_depth=5,
                               # сколько переменных использовать при построении дерева?
                               # sqrt(k)
                               max_features='auto', 
                               # максимальное число конечных узлов. Сейчас - без ограничений
                               max_leaf_nodes=None,
                               # минимальное уменьшение загрязнения 
                               min_impurity_decrease=0.0, 
                               # Устарело
                               min_impurity_split=None,
                               # Дерево обучается по части наблюдений (По какой???)
                               bootstrap=True, 
                               # минимальное число наблюдений в узле потомке
                               min_samples_leaf=1, 
                               # минимальное число наблюдений в узле родителе
                               min_samples_split=2,
                               # параметр используется редко, самостоятельно
                               min_weight_fraction_leaf=0.0, 
                               # объем промежуточных результатов в ходе обучения
                               verbose=1, 
                               # число используемых процессоров
                               # n_jobs=None,
                               # Вычислять out-of-bag ошибку
                               oob_score=True,
                               # использовать результаты предыдущего вызова и нарастить предыдущий лес 
                               warm_start=False,
                               # веса классов для балансировки обучения
                               class_weight=None)

model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.1s finished


RandomForestClassifier(max_depth=5, n_estimators=80, oob_score=True,
                       random_state=42, verbose=1)

In [136]:
# Активируем поиск на решетке GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {  
      'n_estimators': [30, 100, 200],
      'max_depth': [4, 5, 6],
      'min_samples_split': [8, 10, 12, 14]
  }


In [137]:
model2 = RandomForestClassifier(random_state=42, class_weight=None)
grid_search_res = GridSearchCV(model2, param_grid=param_grid, cv=2, error_score='raise', 
                               pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)


In [138]:
grid_search_res.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [4, 5, 6],
                         'min_samples_split': [8, 10, 12, 14],
                         'n_estimators': [30, 100, 200]},
             scoring='accuracy')

In [139]:
grid_search_res.cv_results_

{'mean_fit_time': array([0.04876435, 0.12768793, 0.25049531, 0.03679848, 0.13071775,
        0.2394588 , 0.03642333, 0.1203711 , 0.24076915, 0.03692639,
        0.12193918, 0.24897766, 0.04390502, 0.12975395, 0.25926244,
        0.03733206, 0.12552655, 0.28447032, 0.03480518, 0.13396716,
        0.26913595, 0.04250062, 0.13673222, 0.24799573, 0.04367101,
        0.15540266, 0.27769387, 0.04271984, 0.14470625, 0.27702844,
        0.03936195, 0.1323607 , 0.28959739, 0.04586351, 0.14420879,
        0.25824761]),
 'std_fit_time': array([6.54065609e-03, 4.76837158e-04, 7.07900524e-03, 4.66823578e-04,
        4.60767746e-03, 6.20174408e-03, 1.22904778e-04, 2.59804726e-03,
        1.44290924e-03, 5.33461571e-04, 1.78813934e-05, 1.40736103e-02,
        3.00407410e-04, 4.84836102e-03, 1.70379877e-02, 1.11746788e-03,
        2.79653072e-03, 1.45685673e-02, 4.78029251e-05, 1.12874508e-02,
        9.86957550e-03, 8.00013542e-04, 6.40857220e-03, 1.18851662e-04,
        9.66787338e-05, 4.70066071e-0

In [140]:
# Строим предсказание модели
y_pred_train = grid_search_res.predict(X_train)
y_pred_test = grid_search_res.predict(X_test)

In [141]:
from sklearn import metrics
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,5,6,7
5,125,22,1
6,36,86,9
7,2,22,17


In [142]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=grid_search_res.classes_, columns=grid_search_res.classes_)
conf_mat

,5,6,7
5,512,81,3
6,105,394,8
7,6,75,95


In [143]:
# Строим предсказание модели
y_pred_train2 = grid_search_res.predict_proba(X_train)
y_pred_test2 = grid_search_res.predict_proba(X_test)
part = sum(np.sum(y_pred_test2>0.8, axis =1))/np.shape(y_pred_test2)[0]*100
print('Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8 равна',part,'%')

Доля наблюдений в тестовой выборке, для которых есть класс, вероятность принадлежать которому больше 0.8 равна 9.6875 %


In [144]:
y_pred_train2 = grid_search_res.predict(X_train)
y_pred_test2 = grid_search_res.predict(X_test)
print(metrics.classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

           5       0.84      0.77      0.80       163
           6       0.66      0.66      0.66       130
           7       0.41      0.63      0.50        27

    accuracy                           0.71       320
   macro avg       0.64      0.69      0.65       320
weighted avg       0.73      0.71      0.72       320



In [145]:
best_CV_score = grid_search_res.best_estimator_
test_acc = best_CV_score.score(X_test,y_test)
# начение критерия качества для лучших значений гиперпараметров 
print('Лучшее accuracy на тестовом множестве', test_acc)

Лучшее accuracy на тестовом множестве 0.7125


## Калибровка

In [146]:
from sklearn.calibration import CalibratedClassifierCV

In [147]:
model_sigmoid = CalibratedClassifierCV(grid_search_res, cv=2, method='sigmoid')

In [148]:
model_sigmoid.fit(X_train, y_train)

CalibratedClassifierCV(base_estimator=GridSearchCV(cv=2, error_score='raise',
                                                   estimator=RandomForestClassifier(random_state=42),
                                                   param_grid={'max_depth': [4,
                                                                             5,
                                                                             6],
                                                               'min_samples_split': [8,
                                                                                     10,
                                                                                     12,
                                                                                     14],
                                                               'n_estimators': [30,
                                                                                100,
                                                      

In [149]:
y_pred_train3 = model_sigmoid.predict(X_train)
y_pred_test3 = model_sigmoid.predict(X_test)
print(metrics.classification_report(y_pred_test3, y_test))

              precision    recall  f1-score   support

           5       0.84      0.76      0.79       164
           6       0.60      0.65      0.62       122
           7       0.44      0.53      0.48        34

    accuracy                           0.69       320
   macro avg       0.63      0.64      0.63       320
weighted avg       0.71      0.69      0.70       320

